In [1]:
import numpy as np
from numpy.random import *
from utils import mnist_reader

In [2]:
#画像の読み込み
train_img, train_label = mnist_reader.load_mnist('data', kind='train')
test_img, test_label = mnist_reader.load_mnist('data', kind='t10k')
#画素値を0~1の間の値に変換
train_img = train_img / 255
test_img = test_img / 255
train_img = train_img.astype(np.float32)
#ラベルをone-hot表現で表す
rain_label = [int(x) for x in train_label]
train_label_one_hot = np.identity(10)[train_label].astype(np.float32)
test_label = [int(x) for x in test_label]
test_label_one_hot = np.identity(10)[test_label]

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    e = np.exp(x)
    return e / np.sum(e)

def sigmoid_dash(x):
    return x * (1 - x)

def cross_entropy(y, t):
    E = 0.0
    for i in range(len(t[0])):
        E = E - t[0][i]*np.log(y[0][i])
    return E.astype(np.float32)

In [4]:
#各レイヤで共通な機能を実装
class Layer(object):
    def __init__(self, lr=0.01):
        self.params = {}
        self.grads = {}
        self.lr = lr
        
    def update(self):
        for k in self.params.keys():
            self.params[k] = self.params[k] - self.lr * self.grads[k]
            
    def zerograd(self):
        for k in self.params.keys():
            self.grads[k] = np.zeros(shape = self.params[k].shape, dtype = self.params[k].dtype)
            
    def debug(self):
        for k in self.params.keys():
            print(self.params[k])
            
#順伝播、逆伝播を行う機能を実装
class Sequential:
    def __init__(self, layers=[]):
        self.layers = layers
    #レイヤの追加
    def addlayer(self, layer):
        self.layers.append(layer)
    #順伝播の計算
    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)
        return x
    #逆伝播の計算
    def backward(self, y):
        for l in reversed(self.layers):
            y = l.backward(y)
        return y
    #
    def update(self):
        for l in self.layers:
            l.update()
    #勾配をゼロに
    def zerograd(self):
        for l in self.layers:
            l.zerograd()
            
    def debug(self):
        for l in self.layers:
            l.debug()

class LinearLayer(Layer):
    def __init__(self, input_dim, output_dim):
        super(LinearLayer, self).__init__()
        #正規分布に従った乱数による重みの初期化（Xivierの初期値）
        self.params['W'] = np.random.normal(loc = 0.0, scale = np.sqrt(1.0/input_dim), size = (input_dim, output_dim)).astype(np.float32)
        #バイアスの初期値をゼロに設定
        self.params['b'] = np.zeros(shape = (1, output_dim), dtype = np.float32)
    
    def forward(self, x):
        self.x = x
        return np.dot(x, self.params['W']) + self.params['b']
        
    def backward(self, y):
        self.grads['W'] = np.dot(self.x.T, y)
        self.grads['b'] = y
        #print(self.grads['b'])
        return y*self.params['W']
    
class SigmoidLayer(Layer):
    def __init__(self):
        super(SigmoidLayer, self).__init__()
    
    def forward(self, x):
        z = sigmoid(x)
        self.z = z
        return z
    
    def backward(self, y):
        return  np.array([np.sum(y*sigmoid_dash(self.z).T, axis=1)])
    
class Classfier:
    def __init__(self, model):
        self.model = model
        
    def update(self, x, t):
        self.model.zerograd()
        y = self.model.forward(x)
        prob = softmax(y)
        loss = cross_entropy(prob, t)
        dout = prob - t
        dout = self.model.backward(dout)
        self.model.update()
    
    def test(self, x):
        y = softmax(self.model.forward(x))
        prob = np.array([y[0]])
        return prob

In [5]:
model = Sequential()
model.addlayer(LinearLayer(784,2000))
model.addlayer(SigmoidLayer())
model.addlayer(LinearLayer(2000,500))
model.addlayer(SigmoidLayer())
model.addlayer(LinearLayer(500,10))
classifier = Classfier(model)

In [6]:
for i in range(100):
    rand1 = np.arange(60000)
    shuffle(rand1)
    rand2 = np.arange(10000)
    shuffle(rand2)
    train_img = train_img[rand1,:]
    train_label_one_hot = train_label_one_hot[rand1,:]
    test_img = test_img[rand2]
    test_label_one_hot = np.array(test_label_one_hot[rand2,:])
    for j in range(200):
        x = np.array([train_img[j]])
        t = np.array([train_label_one_hot[j]])
        classifier.update(x, t)
    count = 0
    for j in range(100):
        x = np.array([test_img[j]])
        prob = classifier.test(x)
        if np.argmax(prob[0]) == np.argmax(test_label_one_hot[j]):
            count += 1
    print(i+1)
    print(count/100)

1
0.09
2
0.11
3
0.1
4
0.16
5
0.23
6
0.39
7
0.25
8
0.3
9
0.4
10
0.38
11
0.39
12
0.56
13
0.44
14
0.51
15
0.42
16
0.56
17
0.48
18
0.47
19
0.63
20
0.59
21
0.46
22
0.55
23
0.71
24
0.7
25
0.66
26
0.66
27
0.62
28
0.55
29
0.7
30
0.68
31
0.59
32
0.65
33
0.64
34
0.71
35
0.61
36
0.6
37
0.64
38
0.77
39
0.64
40
0.63
41
0.63
42
0.67
43
0.73
44
0.64
45
0.72
46
0.65
47
0.66
48
0.74
49
0.7
50
0.75
51
0.66
52
0.67
53
0.65
54
0.66
55
0.59
56
0.78
57
0.75
58
0.69
59
0.65
60
0.65
61
0.75
62
0.75
63
0.68
64
0.7
65
0.69
66
0.77
67
0.75
68
0.69
69
0.81
70
0.77
71
0.64
72
0.74
73
0.6
74
0.66
75
0.75
76
0.67
77
0.75
78
0.75
79
0.7
80
0.7
81
0.71
82
0.75
83
0.71
84
0.76
85
0.72
86
0.67
87
0.72
88
0.69
89
0.68
90
0.65
91
0.66
92
0.76
93
0.67
94
0.81
95
0.75
96
0.76
97
0.78
98
0.76
99
0.68
100
0.73
